In [ ]:
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
# company info
def page_participation(page = 1):
    url = "https://unglobalcompact.org/what-is-gc/participants?page=" + str(page)
    # Requests URL and get response object
    response = requests.get(url)

    # Parse text obtained
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all hyperlinks present on webpage
    links = soup.find_all('a')

    # From all links check for pdf link and
    # if present download file
    df = pd.DataFrame()
    for link in links:
        if ('/what-is-gc/participants/' in link.get('href', [])):
            link_company = "https://unglobalcompact.org" + link.get('href')
            page = requests.get(link_company)
            # Create a BeautifulSoup object
            soup = BeautifulSoup(page.text, 'html.parser')
            title = soup.find_all("title")[0].text
            cat = [i.text.replace(":", "") for i in soup.find_all("dt")]
            para = [i.text for i in soup.find_all("dd")]
            tmp = pd.DataFrame(para).T
            tmp.columns = cat
            tmp["link"] = link_company
            df = pd.concat([df, tmp])
    return df

In [ ]:
#df_ = page_participation(page = 1)
for page in range(610, 2029):
    tmp_df = page_participation(page = page)
    tmp_df["page"] = page
    df_ = pd.concat([df_, tmp_df])
    df_.to_csv("company_scrape.csv")
    print(page)

In [ ]:
df_ = pd.read_csv("./data/company_scrape.csv")

In [ ]:
url_init = set(df_["link"].tolist())

In [ ]:
def retrieve_links(links_comp, url_init, Country, Sector):
    df_pdfs = pd.DataFrame()
    for i in range(len(links_comp)): 
        upper_link = links_comp[i].get("href")
        upper_content = links_comp[i].contents
        if upper_link[:2] == "//":
            url = upper_link.replace("//", "")
        elif "http" in upper_link:
            url = upper_link
        else:
            url = "https://unglobalcompact.org" + upper_link
        upper_link = url

        if ("communication" in str(upper_content).lower()) or ("report" in str(upper_content).lower()) or ("review" in str(upper_content).lower()) or ("cop" in str(upper_content).lower()):
            dct = {"upper_link": upper_link, "upper_content": upper_content, "pdf_link":[''],"pdf_content": [""]}
            #print("init url", url, "\n\ninit content ", upper_content)
            try:
                response = requests.get(upper_link)

                # Parse text obtained
                soup = BeautifulSoup(response.text, 'html.parser')
                links  = soup.find_all('a')
                lst_pdfs = []
                lst_contents = []
                for l in links:
                    pdf_link = l.get("href")
                    content = l.contents
                    try:
                        if "pdf" in pdf_link:
                            lst_pdfs.append(pdf_link)
                            lst_contents.append(content)
                    except: pass
                dct["pdf_link"] = lst_pdfs 
                dct["pdf_content"] = content
                tmp = pd.DataFrame(dct)
                df_pdfs = pd.concat([df_pdfs, tmp])
            except:
                tmp = pd.DataFrame.from_dict(dct, orient='index').T
                df_pdfs = pd.concat([df_pdfs, tmp])
            df_pdfs["link_company"] = url_init
            df_pdfs["Country"] = Country
            df_pdfs["Sector"] = Sector
        else: pass #print("pass: ", url)
    return df_pdfs

In [ ]:
pdfs_link_done = pd.read_csv("./data/pdf_links_collection.csv") # if any previous search has been saved
missed = set(df_.link).difference(set(pdfs_link_done.link_company.unique()))
missed_df = df_.set_index("link").loc[list(missed)].reset_index()

In [ ]:
pdfs_link = pd.DataFrame()
for i in range(len(missed_df)):
    url_init, Country, Sector= missed_df.iloc[i*(-1)][["link", "Country", "Sector"]]
    response = requests.get(url_init)

    # Parse text obtained
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all hyperlinks present on webpage
    links_comp = soup.find_all('a')
    tmp = retrieve_links(links_comp, url_init, Country, Sector)
    pdfs_link = pd.concat([pdfs_link, tmp])
    print(i)
    if i%10 ==0:
        pdfs_link.to_csv("pdf_links_collection_R2.csv", index = False)

In [ ]:
ISO3 = pd.read_csv("./data/iso_3.csv", sep=",", encoding='cp1252')

In [ ]:
import re
pdfs_link["company_name"] = pdfs_link.link_company.apply(lambda x: x.split("/")[-1].replace("-", " "))
pdfs_link["company_id"] = pdfs_link["company_name"].apply(lambda x: x.split()[0])
pdfs_link["company_id"].nunique()
pdfs_link["year"] = pdfs_link.upper_content.apply(lambda x: re.findall(r'\d+', str(x)))
pdfs_link["year"] = [int(y[0]) if y else None for y in pdfs_link["year"]]
pdfs_link["year_links"] = pdfs_link.pdf_link.apply(lambda x: re.findall(r'\d+', " ".join(x)))

In [ ]:
target_2023 = pd.merge(target_2023, ISO3, left_on = "Country", right_on = "country", how = "left")
target_2023[target_2023["code"].isna()].Country.unique()
target_2023["file_id"] = target_2023.company_id.apply(lambda x: "2023_" + str(x))

In [ ]:
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.options import Options
import time
from pathlib import Path
import re

def retrive_link(link, file_id, ISO, idx):
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument("--headless")
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(link)
    time.sleep(6) # any number > 3 should work fine
    html = driver.page_source
    
    try:
        prefix = 'https://cop-report.'
        pdf_link = re.findall(prefix + '([\w\-\./]+)', html) 
        response = requests.get(prefix + pdf_link[0])
        filename = Path('pdf/2023/' + file_id + "_" + ISO + '.pdf')
        filename.write_bytes(response.content)
    except: 
        print("Error: ", file_id, link, idx)

In [ ]:
for idx in range(len(target_2023)):
    row = target_2023.iloc[idx]
    
    link = row["upper_link"]
    file_id = row["file_id"]
    ISO = row["code"]
    
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    links_comp = soup.find_all('a')
    quota1 = False
    try:
        quota1 = links_comp[0].contents[0] == 'Reload'
    except: 
        pass
    if quota1 or (len(links_comp) == 0):
        retrive_link(link, file_id, ISO, idx)
    else:
        for link in links_comp:
            if ('.pdf' in link.get('href', [])):
                print("Downloading file: ", idx)
                if "http" in link.get('href'):
                    response = requests.get(link.get('href'))
                else:
                    response = requests.get("http:" + link.get('href'))

                filename = Path('pdf/2023/' + file_id + "_" + ISO + '.pdf')
                filename.write_bytes(response.content)
                print("File ", idx, " downloaded")

In [ ]:
# pdf to text
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pytesseract 
# To remove the additional created files
import os

In [ ]:
# Create a function to extract text
import pdfplumber
# To extract the images from the PDFs
from PIL import Image
from pdf2image import convert_from_path
# To perform OCR to extract text from images 
import pytesseract 
# To remove the additional created files
import os

def text_extraction(element):
    # Extracting the text from the in-line text element
    line_text = element.get_text()
    
    # Find the formats of the text
    # Initialize the list with all the formats that appeared in the line of text
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character
                    line_formats.append(character.fontname)
                    # Append the font size of the character
                    line_formats.append(character.size)
    # Find the unique font sizes and names in the line
    format_per_line = list(set(line_formats))
    
    # Return a tuple with the text in each line along with its format
    return (line_text, format_per_line)

# Create a function to crop the image elements from PDFs
def crop_image(element, pageObj):
    # Get the coordinates to crop the image from the PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1] 
    # Crop the page using coordinates (left, bottom, right, top)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    # Save the cropped page to a new PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    # Save the cropped PDF to a new file
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

# Create a function to convert the PDF to images
def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = "PDF_image.png"
    image.save(output_file, "PNG")

# Create a function to read text from images
def image_to_text(image_path):
    # Read the image
    img = Image.open(image_path)
    # Extract the text from the image
    text = pytesseract.image_to_string(img)
    return text

# Extracting tables from the page

def extract_table(pdf_path, page_num, table_num):
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    table_page = pdf.pages[page_num]
    # Extract the appropriate table
    table = table_page.extract_tables()[table_num]
    return table

# Convert table into the appropriate format
def table_converter(table):
    table_string = ''
    # Iterate through each row of the table
    for row_num in range(len(table)):
        row = table[row_num]
        # Remove the line breaker from the wrapped texts
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Convert the table into a string 
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Removing the last line break
    table_string = table_string[:-1]
    return table_string

In [ ]:
def pdf_to_txt(pdf_path):
    # create a PDF file object
    pdfFileObj = open(pdf_path, 'rb')
    # create a PDF reader object
    pdfReaded = PyPDF2.PdfReader(pdfFileObj)

    # Create the dictionary to extract text from each image
    text_per_page = {}
    # We extract the pages from the PDF
    for pagenum, page in enumerate(extract_pages(pdf_path)):
        if pagenum%5 == 0:
            print(pagenum)
        # Initialize the variables needed for the text extraction from the page
        pageObj = pdfReaded.pages[pagenum]
        page_text = []
        line_format = []
        text_from_images = []
        text_from_tables = []
        page_content = []
        # Initialize the number of the examined tables
        table_num = 0
        first_element= True
        table_extraction_flag= False
        # Open the pdf file
        pdf = pdfplumber.open(pdf_path)
        # Find the examined page
        page_tables = pdf.pages[pagenum]
        # Find the number of tables on the page
        tables = page_tables.find_tables()


        # Find all the elements
        page_elements = [(element.y1, element) for element in page._objs]
        # Sort all the elements as they appear in the page 
        page_elements.sort(key=lambda a: a[0], reverse=True)
        
        if_text = False
        # Find the elements that composed a page
        for i, component in enumerate(page_elements):
            # Extract the position of the top side of the element in the PDF
            pos= component[0]
            # Extract the element of the page layout
            element = component[1]
            
            # Check if the element is a text element
            if isinstance(element, LTTextContainer):
                # Check if the text appeared in a table
                if table_extraction_flag == False:
                    # Use the function to extract the text and format for each text element
                    (line_text, format_per_line) = text_extraction(element)
                    # Append the text of each line to the page text
                    page_text.append(line_text)
                    # Append the format for each line containing text
                    line_format.append(format_per_line)
                    page_content.append(line_text)
                else:
                    # Omit the text that appeared in a table
                    pass
            #
            # Check the elements for images
            if (isinstance(element, LTFigure)) & (pagenum >2):
                # Crop the image from the PDF
                crop_image(element, pageObj)
                # Convert the cropped pdf to an image
                convert_to_images('cropped_image.pdf')
                # Extract the text from the image
                image_text = image_to_text('PDF_image.png')
                text_from_images.append(image_text)
                page_content.append(image_text)
                # Add a placeholder in the text and format lists
                page_text.append('image')
                line_format.append('image')
            
        # Create the key of the dictionary
        dctkey = 'Page_'+str(pagenum)
        # Add the list of list as the value of the page key
        text_per_page[dctkey]= page_content
    os.remove('cropped_image.pdf')
    os.remove('PDF_image.png')
    # Closing the pdf file object
    pdfFileObj.close()
    return text_per_page

In [ ]:
import os

def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return file_info.st_size/1024

In [ ]:
import json
from tika import parser
for f in os.listdir("./pdf/2023/"):
    print(f)
    try:
        path_pdf = "./pdf/2023/" + f
        # Deleting the additional files created
        text_per_page = parser.from_file(path_pdf)
        name = f.replace("pdf", "txt")
        with open('txt/2023/tika_' + name, 'w') as file:
            file.write(json.dumps(text_per_page["content"]))

        file_path = 'txt/2023/tika_' + name
        size = file_size(file_path)
        if size > 50:
            os.remove(path_pdf)
        else:
            print("undeleted: ", path_pdf)
            destination = "./pdf/done/" + f
            os.rename(path_pdf, destination)
    except:
        print("failed: ", path_pdf)
        destination = "./pdf/failed/" + f
        os.rename(path_pdf, destination)

In [ ]:
import langdetect
import ast
def preprocess_text(text):
    lst_sent = []
    text = text.encode().decode('unicode-escape')
    text = text.replace("-\n", "").split("\n\n")

    for sent in text:
        if len(sent) > 0:
            sent = sent.replace("\n", " ")
            numbe_per = sum([alph.isnumeric() for alph in sent])/len(sent)
            if (numbe_per<0.9):
                lst_sent.append(sent)
            else:
                pass
    return lst_sent

In [ ]:
# meta is the meta information collected from previous stage;

In [ ]:
for idx, row in meta.iterrows():
    try:
        file = "./txt/2023/" + row["pdf_id"].split(".")[0] + ".txt"
        with open(file,"r") as f:
            tmp = f.read()
        #tmp = preprocess_text(tmp)
    except:
        try:
            file = "./txt/2023/" + row["pdf_id"].split(".")[0] + "_tika.txt"
            with open(file,"r") as f:
                tmp = f.read()
            #tmp = preprocess_text(tmp)
        except:
            tmp = None
    meta.loc[idx, "text"] =tmp